In [8]:
import argparse
import time

import torch
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

from models import GCN
from sampler import Sampler_FastGCN, Sampler_ASGCN
from utils import load_data, get_batches, accuracy
from utils import sparse_mx_to_torch_sparse_tensor




dataset = 'cora'
model = 'Fast'
test_gap = 10
hidden = 16
fastmode = False
seed = 123
epochs = 100
lr = 0.01
weight_decay = 5e-4
dropout = 0.0
batchsize = 512

def train(train_ind, train_labels, batch_size, train_times):
    t = time.time()
    model.train()
    for epoch in range(train_times):
        for batch_inds, batch_labels in get_batches(train_ind,
                                                    train_labels,
                                                    batch_size):
            sampled_feats, sampled_adjs, var_loss = model.sampling(
                batch_inds)
            optimizer.zero_grad()
            output = model(sampled_feats, sampled_adjs)
            loss_train = loss_fn(output, batch_labels) + 0.5 * var_loss
            acc_train = accuracy(output, batch_labels)
            loss_train.backward()
            optimizer.step()
    # just return the train loss of the last train epoch
    return loss_train.item(), acc_train.item(), time.time() - t


def test(test_adj, test_feats, test_labels, epoch):
    t = time.time()
    model.eval()
    outputs = model(test_feats, test_adj)
    loss_test = loss_fn(outputs, test_labels)
    acc_test = accuracy(outputs, test_labels)

    return loss_test.item(), acc_test.item(), time.time() - t



    
adj, features, adj_train, train_features, y_train, y_test, test_index = \
    load_data(dataset)

layer_sizes = [1024, 1024]
input_dim = features.shape[1]
train_nums = adj_train.shape[0]
test_gap = test_gap
nclass = y_train.shape[1]

np.random.seed(seed)
torch.manual_seed(seed)

torch.cuda.manual_seed(seed)

device = torch.device("cuda")
print("use cuda")

# data for train and test
features = torch.FloatTensor(features).to(device)
train_features = torch.FloatTensor(train_features).to(device)
y_train = torch.LongTensor(y_train).to(device).max(1)[1]

test_adj = [adj, adj[test_index, :]]
test_feats = features
test_labels = y_test
test_adj = [sparse_mx_to_torch_sparse_tensor(cur_adj).to(device)
            for cur_adj in test_adj]
test_labels = torch.LongTensor(test_labels).to(device).max(1)[1]

# init the sampler

sampler = Sampler_FastGCN(None, train_features, adj_train,
                                input_dim=input_dim,
                                layer_sizes=layer_sizes,
                                device=device)



# init model, optimizer and loss function
model = GCN(nfeat=features.shape[1],
            nhid=hidden,
            nclass=nclass,
            dropout=dropout,
            sampler=sampler).to(device)
optimizer = optim.Adam(model.parameters(),
                        lr=lr, weight_decay=weight_decay)
loss_fn = F.nll_loss
# loss_fn = torch.nn.CrossEntropyLoss()

# train and test
for epochs in range(0, epochs // test_gap):
    train_loss, train_acc, train_time = train(np.arange(train_nums),
                                                y_train,
                                                batchsize,
                                                test_gap)
    test_loss, test_acc, test_time = test(test_adj,
                                            test_feats,
                                            test_labels,
                                            epochs)
    print(f"epchs:{epochs * test_gap}~{(epochs + 1) * test_gap - 1} "
            f"train_loss: {train_loss:.3f}, "
            f"train_acc: {train_acc:.3f}, "
            f"train_times: {train_time:.3f}s "
            f"test_loss: {test_loss:.3f}, "
            f"test_acc: {test_acc:.3f}, "
            f"test_times: {test_time:.3f}s")

torch.save(model.state_dict(), "fastGCN.pt")


use cuda
epchs:0~9 train_loss: 1.706, train_acc: 0.495, train_times: 0.480s test_loss: 1.759, test_acc: 0.403, test_times: 0.003s
epchs:10~19 train_loss: 1.330, train_acc: 0.734, train_times: 0.278s test_loss: 1.390, test_acc: 0.654, test_times: 0.002s
epchs:20~29 train_loss: 1.086, train_acc: 0.799, train_times: 0.256s test_loss: 1.134, test_acc: 0.732, test_times: 0.002s
epchs:30~39 train_loss: 0.885, train_acc: 0.826, train_times: 0.195s test_loss: 0.981, test_acc: 0.793, test_times: 0.001s
epchs:40~49 train_loss: 0.872, train_acc: 0.870, train_times: 0.195s test_loss: 0.874, test_acc: 0.832, test_times: 0.002s
epchs:50~59 train_loss: 0.726, train_acc: 0.891, train_times: 0.262s test_loss: 0.798, test_acc: 0.834, test_times: 0.001s
epchs:60~69 train_loss: 0.724, train_acc: 0.875, train_times: 0.199s test_loss: 0.746, test_acc: 0.843, test_times: 0.002s
epchs:70~79 train_loss: 0.688, train_acc: 0.859, train_times: 0.189s test_loss: 0.707, test_acc: 0.857, test_times: 0.001s
epchs:80~

In [23]:
train_nums = adj_train.shape[0]
train_nums

1208

In [26]:
adj_train.shape[1]

1208

In [25]:
adj_train.shape[0]

1208

In [9]:
import argparse
import time

import torch
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

from models import GCN
from sampler import Sampler_FastGCN, Sampler_ASGCN
from utils import load_data, get_batches, accuracy
from utils import sparse_mx_to_torch_sparse_tensor




dataset = 'cora'
model = 'Fast'
test_gap = 10
hidden = 16
fastmode = False
seed = 123
epochs = 100
lr = 0.01
weight_decay = 5e-4
dropout = 0.0
batchsize = 512

def train(train_ind, train_labels, batch_size, train_times):
    t = time.time()
    model.train()
    for epoch in range(train_times):
        for batch_inds, batch_labels in get_batches(train_ind,
                                                    train_labels,
                                                    batch_size):
            sampled_feats, sampled_adjs, var_loss = model.sampling(
                batch_inds)
            optimizer.zero_grad()
            output = model(sampled_feats, sampled_adjs)
            loss_train = loss_fn(output, batch_labels) + 0.5 * var_loss
            acc_train = accuracy(output, batch_labels)
            loss_train.backward()
            optimizer.step()
    # just return the train loss of the last train epoch
    return loss_train.item(), acc_train.item(), time.time() - t


def test(test_adj, test_feats, test_labels, epoch):
    t = time.time()
    model.eval()
    outputs = model(test_feats, test_adj)
    loss_test = loss_fn(outputs, test_labels)
    acc_test = accuracy(outputs, test_labels)

    return loss_test.item(), acc_test.item(), time.time() - t



    
adj, features, adj_train, train_features, y_train, y_test, test_index = \
    load_data(dataset)

In [19]:
len(features)

2708

In [17]:
len(train_features)

1208

In [18]:
len(y_train)

1208